# Building custom interfaces

This notebook will demonstrate how to create custom interactive interfaces to visualize a data set with `plopp`.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc
import numpy as np

The data is a two-dimensional data array, where the values are generated using a sine function.
We also add a small amount of random noise to the values.

In [ ]:
nx = 200
ny = 150

x = np.arange(float(nx))
y = np.arange(float(ny))
noise = np.random.random((ny, nx))
z = 3.0 * np.sin(np.sqrt(x**2 + y.reshape(ny, 1) ** 2) / 10.0) + noise + 300.0

da = sc.DataArray(
    data=sc.array(dims=['y', 'x'], values=z, unit='K'),
    coords={
        'x': sc.array(dims=['x'], values=x, unit='m'),
        'y': sc.array(dims=['y'], values=y, unit='m'),
    },
)

da

## A set of connected nodes

In `plopp`, think of each element in your interface as a set of interconnected nodes in a graph.
Each node can have parent nodes, children nodes, and also views attached to them (e.g. figures).

At the most basic level, a graph will contain a node (white rectangle) that provides the input data,
and a view (grey ellipse) which will be a figure to display the data visually.
Note that the figure takes as input the `data_node`.

In [ ]:
data_node = pp.Node(da)

fig = pp.imagefigure(data_node)

pp.show_graph(data_node)  # display the graph

When the data in the input node changes, the view is notified about the change.
It requests new data from its parent node and updates the visuals on the figure.

The figure can directly be displayed in the notebook:

In [ ]:
fig

### Nodes are callables

Nodes in the graph have to be constructed from callables.
When a view requests data from a parent node, the callable is called.
Typically, the callable will be a function that takes in a data array as an input,
and returns a data array as an output.

Keeping your inputs and outputs as data arrays is useful because figure views will only accept data arrays as input.
That said, nodes that produce other outputs are very common, for example when using interactive widgets.

In the small example above, the node at the top of the graph has no parents,
and its callable is simply a `lambda` function with no arguments that just returns the input data.

Calling any node will return the output from its internal callable
(this is very similar to [Dask's](https://docs.dask.org/en/stable/delayed.html) `delayed.compute()` method).
In our example above, calling `data_node` will simply return the initial data array

In [ ]:
data_node()

But for more complex graphs, the call will walk the tree, requesting all the pieces of data it needs to compute the final result.

## Expanding the graph

Next, say we wish to add a gaussian smoothing step in our graph, before showing the data on the figure.
We start with the same `data_node`, but add a second node that performs the smoothing operation before attaching the figure.
Because the `gaussian_filter` function requires a kernel width `sigma` as input, we set it to 5 via a keyword argument
(note here that it is not necessary to wrap it into a `Node`, this will automatically be handled internally).

In [ ]:
from scipp.scipy.ndimage import gaussian_filter

data_node = pp.Node(da)

smooth_node = pp.Node(gaussian_filter, data_node, sigma=5)

fig = pp.imagefigure(smooth_node)

The resulting graph has two input nodes (one for the data array and one for the kernel width), a smoothing node, and a figure:

In [ ]:
pp.show_graph(data_node)

And the resulting figure displays the smoothed data:

In [ ]:
fig

## Adding interactive widgets

In the example above, the kernel size `sigma` for the gaussian smoothing was frozen to `5`.
But we would actually want to control this via a slider widget.

In this case, the smoothing node now needs two inputs: the raw data, and the `sigma`.
It gets the raw data from the `data_node`, and the `sigma` from a `widget_node`,
which is coupled to a slider from the `ipywidgets` library.

In [ ]:
import ipywidgets as ipw

data_node = pp.Node(da)

slider = ipw.IntSlider(min=1, max=20)
slider_node = pp.widget_node(slider)

smooth_node = pp.Node(gaussian_filter, data_node, sigma=slider_node)

fig = pp.imagefigure(smooth_node)

As expected, the smoothing node now has a widget as one of its parent nodes instead of the fixed-value input node:

In [ ]:
pp.show_graph(fig)

And we can display the figure and the slider inside the same container:

In [ ]:
ipw.VBox([slider, fig])

When a change occurs in one of the nodes, all the nodes below it in the graph are notified about the change (the children nodes receive a notification, and they, in turn, notify their own children).
It is then up to each view to decide whether they are interested in the notification or not (usually, most views are interested in all notifications from parents).
If they are, they request data from their parent nodes, which in turn request data from their parents, and so on, until the request has reached the top of the graph.

As a result, when the slider is dragged, the smoothing node gets notified and tells the figure that a change has occurred.
The figure tells `smooth_node` that it wants updated data.
`smooth_node` asks nodes `data_node` and `slider_node` for their data.
`data_node` returns the raw data, while `slider_node` returns the integer value for the kernel size.
`smooth_node` then simply sends the inputs to the `gaussian_filter` function, and forwards the result to the figure.

<div class="alert alert-warning">

*Warning*

The figure will not update when dragging the slider in the documentation pages.
This will only work inside a Jupyter notebook.
    
</div>

## Multiple views

To go one step further,
we now wish to add a one-dimensional figure that will display the sum of the two-dimensional data along the vertical dimension.
On this figure, we would like to display both the original (unsmoothed) data, as well as the smoothed data.

In [ ]:
data_node = pp.Node(da)

slider = ipw.IntSlider(min=1, max=20, value=10)
slider_node = pp.widget_node(slider)

smooth_node = pp.Node(gaussian_filter, data_node, sigma=slider_node)

fig2d = pp.imagefigure(smooth_node)

# Sum the raw data along the vertical dimension
sum_raw = pp.Node(sc.sum, data_node, dim='y')
# Sum the smoothed data along the vertical dimension
sum_smoothed = pp.Node(sc.sum, smooth_node, dim='y')
# Give two nodes to a figure to display both on the same axes
fig1d = pp.linefigure(sum_raw, sum_smoothed)

We check the graph again to make sure that the one-dimensional figure has two inputs,
and that both are performing a sum along the `y` dimension.

In [ ]:
pp.show_graph(slider_node)

In [ ]:
ipw.VBox([slider, fig2d, fig1d])

Because the slider only affects the smoothing part of the graph,
only the orange markers will update when we drag the slider.

## Multiple controls and the `node` decorator

In this section, masks will be added to the raw data,
and a widget made of checkboxes will be used to toggle the masks on and off.

We make a slightly different interface, where we remove the gaussian smoothing,
and instead use the slider to slice rows in the input data.

In [ ]:
da.masks['close_to_300'] = abs(da.data - sc.scalar(300.0, unit='K')) < sc.scalar(
    1.0, unit='K'
)
da.masks['large_x'] = da.coords['x'] > sc.scalar(150.0, unit='m')
da

The data node remains the same:

In [ ]:
data_node = pp.Node(da)

Next, we create a `Checkboxes` widget which takes in a set of keys/strings as input.
It will create a checkbox per entry.

In [ ]:
from plopp.widgets import Checkboxes, Box

masks_widget = Checkboxes(da.masks.keys())
masks_widget

The `.value` of the widget simply contains the values for the individual checkboxes:

In [ ]:
masks_widget.value

We make a node from the checkboxes widget using `widget_node` once again.

We can supply custom functions to the `Node` constructor,
but it is often useful to decorate our functions with the `plopp.node` decorator,
and then call the functions directly.

We therefore define a function that will hide masks depending on the checkboxes values,
and decorate it with `@pp.node` so that it can accept `data_node` and `masks_node` as inputs.

In [ ]:
masks_node = pp.widget_node(masks_widget)


@pp.node
def hide_masks(data_array, masks):
    out = data_array.copy(deep=False)
    for name, value in masks.items():
        if name in out.masks and (not value):
            del out.masks[name]
    return out


hide_node = hide_masks(data_array=data_node, masks=masks_node)

We then connect the slider to a slicing function,
and add that below the `hide_node` in the graph:

In [ ]:
@pp.node
def slice_y(data_array, index):
    return data_array['y', index]


slider = ipw.IntSlider(min=0, max=da.sizes['y'] - 1)
slider_node = pp.widget_node(slider)

slice_node = slice_y(hide_node, index=slider_node)

pp.show_graph(masks_node)

Finally, we add an image figure to the `hide_node`,
and a line figure after the slicing has been performed.

In [ ]:
fig2d = pp.imagefigure(hide_node)
fig1d = pp.linefigure(slice_node)

pp.show_graph(masks_node)

In [ ]:
Box([[slider, masks_widget], fig2d, fig1d])

Note that the masks update on both the 1D and 2D figures,
because the checkboxes widget lies at the very top of the graph.